In [1]:
import pandas as pd
import gensim
from gensim.models import FastText
import nltk
from nltk.stem import WordNetLemmatizer
import re

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords

In [3]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [4]:
data_loc = 'data/'

In [5]:
data1 = pd.read_csv(data_loc + 'winemag-data_first150k.csv', index_col=0)
data2 = pd.read_csv(data_loc + 'winemag-data-130k-v2.csv', index_col=0)

In [6]:
data2 = data2.iloc[:75000]

In [7]:
#Panen andmed kokku
dataset = pd.concat([data1,data2])

In [8]:
dataset.reset_index(drop=True)

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225925,Austria,"There is a slight, aromatic lift on the nose t...",NaN,88,14.0,Kremstal,NaN,NaN,Grüner Veltliner,Winzer Krems,Anne Krebiehl MW,@AnneInVino,Winzer Krems 2015 Grüner Veltliner (Kremstal)
225926,New Zealand,"This plump, dry-style Pinot Gris features plen...",NaN,88,15.0,Marlborough,NaN,NaN,Pinot Gris,Yealands,Joe Czerwinski,@JoeCz,Yealands 2016 Pinot Gris (Marlborough)
225927,Italy,"Made with 80% Sangiovese and 20% Syrah, this h...",NaN,88,22.0,Tuscany,Toscana,NaN,Red Blend,Ziobaffa,Kerin O’Keefe,@kerinokeefe,Ziobaffa 2015 Red (Toscana)
225928,US,One of the first bottles to bear the name of t...,Single Vineyard,88,30.0,Idaho,NaN,NaN,Syrah,3 Horse Ranch Vineyards,Sean P. Sullivan,@wawinereport,3 Horse Ranch Vineyards 2014 Single Vineyard S...


In [9]:
#Vaatan, mis toimub
dataset.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,NaN,NaN,NaN
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,NaN,NaN,NaN
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,NaN,NaN,NaN
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,NaN,NaN,NaN
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,NaN,NaN,NaN


In [10]:
desc = dataset.description

In [11]:
reviews_lower = [x.lower() for x in desc]

In [12]:
reviews_lower[3]

"this spent 20 months in 30% new french oak, and incorporates fruit from ponzi's aurora, abetina and madrona vineyards, among others. aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. drink now through 2032."

In [13]:
#Puhastan teksti sodist, aga võite vaielda, et äkki ikkagi on vaja mingit tüüpi arve säilitada (nt aastaarvud)
cleaned_reviews = [re.sub('[^a-zA-Z]', ' ', x) for x in reviews_lower]
cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]

In [14]:
cleaned_reviews[3]

'this spent months in new french oak and incorporates fruit from ponzi s aurora abetina and madrona vineyards among others aromatic dense and toasty it deftly blends aromas and flavors of toast cigar box blackberry black cherry coffee and graphite tannins are polished to a fine sheen and frame a finish loaded with dark chocolate and espresso drink now through '

In [15]:
reviews = [nltk.sent_tokenize(x.strip()) for x in cleaned_reviews]

In [16]:
reviews = [y for x in reviews for y in x]

In [17]:
reviews[3]

'this spent months in new french oak and incorporates fruit from ponzi s aurora abetina and madrona vineyards among others aromatic dense and toasty it deftly blends aromas and flavors of toast cigar box blackberry black cherry coffee and graphite tannins are polished to a fine sheen and frame a finish loaded with dark chocolate and espresso drink now through'

In [18]:
words = [nltk.word_tokenize(x) for x in reviews]

In [19]:
for i in range(len(words)):
    words[i] = [lemmatizer.lemmatize(w) for w in words[i] if w not in stop_words and len(w) > 1]
    # words[i] = [w for w in words[i] if w not in stop_words]
# words = [[w for w in words[i] if w not in stop_words] for i in range(len(words))]

In [20]:
#Põhimõtteliselt on korpus valmis
words[3]

['spent',
 'month',
 'new',
 'french',
 'oak',
 'incorporates',
 'fruit',
 'ponzi',
 'aurora',
 'abetina',
 'madrona',
 'vineyard',
 'among',
 'others',
 'aromatic',
 'dense',
 'toasty',
 'deftly',
 'blend',
 'aroma',
 'flavor',
 'toast',
 'cigar',
 'box',
 'blackberry',
 'black',
 'cherry',
 'coffee',
 'graphite',
 'tannin',
 'polished',
 'fine',
 'sheen',
 'frame',
 'finish',
 'loaded',
 'dark',
 'chocolate',
 'espresso',
 'drink']

In [21]:
# Mudel, aga epohhe võiks rohkem olla (100)
# model = gensim.models.Word2Vec(words, vector_size = 200 , window = 4 , min_count = 1,
#                                seed = 1000, hs = 0, negative = 10,workers=16, epochs = 25)
model = gensim.models.FastText(words, vector_size = 200 , window = 4 , min_count = 1,
                               seed = 1000, hs = 0, negative = 10,workers=16, epochs = 25)

In [22]:
model.save("fastText.model")

In [23]:
#Kümme kõige lähedasemat sõna sõnale "magus"
model.wv.most_similar('sweet')

[('sweeet', 0.9062697291374207),
 ('sweety', 0.8742073178291321),
 ('sweetbut', 0.8738280534744263),
 ('sweex', 0.8331032991409302),
 ('sweetshop', 0.8142277002334595),
 ('sweetzer', 0.7973511815071106),
 ('sweetish', 0.7814955711364746),
 ('sweetie', 0.7744488716125488),
 ('ultrasweet', 0.7652175426483154),
 ('sweetnees', 0.757911205291748)]

In [24]:
#Kümme kõige lähedasemat sõna sõnale "kuiv"
model.wv.most_similar('dry')

[('dryly', 0.7361078858375549),
 ('nedry', 0.7342773675918579),
 ('tawdry', 0.7269936800003052),
 ('drysack', 0.7043814659118652),
 ('dryer', 0.6830528378486633),
 ('wizardry', 0.6430472135543823),
 ('ultradry', 0.6318948864936829),
 ('dryish', 0.6156303882598877),
 ('drylands', 0.5800159573554993),
 ('laundry', 0.5367393493652344)]

Päris asjani on tükk maad veel. Mu arust võiks jõuda selleni, et saaks veinisoovitusi nii karakterit kirjeldavate märksõnade kui veininime järgi (ehk mis veinid oleksid konkreetsele veinile kõige sarnasemad). Word2veci saab ära kasutada selles mõttes, et märksõna järgi otsides ei pea istuma me kinni ühe kindla sõna küljes (nt kui otsime kuiva veini, siis me ei saa vastu ainult neid, mille kirjelduses on konkreetselt sõna "kuiv"), vaid saame võtta arvesse ka selle sünonüüme/semantiliselt lähedasi sõnu. Siinse katsetuse põhjal saame ehk algatada arutelu?